Version 1

In [ ]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import pyrealsense2 as rs
import datetime as dt
import time
import csv
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV , cross_val_score
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier , VotingClassifier
from sklearn.metrics import accuracy_score # Accuracy metrics 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.inspection import permutation_importance
import logging
import pickle 
import rospy
import moveit_commander
import sys
from math import pi
import moveit_msgs.msg
import geometry_msgs.msg
from geometry_msgs.msg import TwistStamped
from subprocess import *
import serial
from time import sleep




Open camera to count keypoint

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import os
import csv
from sklearn.cluster import KMeans
import pyrealsense2 as rs
import pandas as pd

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

font = cv2.FONT_HERSHEY_COMPLEX
org = (20, 100)
fontScale = 0.5
thickness = 1
color = (0, 150, 255)

# RealSense camera setup
realsense_ctx = rs.context()
connected_devices = [realsense_ctx.devices[i].get_info(rs.camera_info.serial_number) for i in range(len(realsense_ctx.devices))]

print("Connected Devices:", connected_devices)

camera_input = input("Select camera (0 or 1): ")
if camera_input.isdigit() and int(camera_input) < len(connected_devices):
    camera_id = connected_devices[int(camera_input)]
else:
    print("Invalid selection. Using default camera.")
    camera_id = connected_devices[0]

print("Using camera:", camera_id)

# Configure RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_device(camera_id)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
profile = pipeline.start(config)

align_to = rs.stream.color
align = rs.align(align_to)

depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
clipping_distance = 2 / depth_scale  # 2 meters

# Data Storage
collected_data = []
labels = []

def normalize_hand_keypoints(landmarks):
    """ Normalize hand keypoints based on the hand size and position """
    keypoints = np.array([[lm.x, lm.y, lm.z] for lm in landmarks.landmark])
    
    # Compute reference scale (wrist to index finger tip)
    wrist = keypoints[0]
    index_tip = keypoints[8]
    scale = np.linalg.norm(wrist - index_tip)  

    return ((keypoints - wrist) / scale).flatten()  # Flatten to 1D array

while True:
    frames = pipeline.wait_for_frames()
    aligned_frames = align.process(frames)
    depth_frame = aligned_frames.get_depth_frame()
    color_frame = aligned_frames.get_color_frame()

    if not depth_frame or not color_frame:
        continue

    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())
    color_image = cv2.flip(color_image, 1)

    image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)

    result = hands.process(image)
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            keypoints = normalize_hand_keypoints(hand_landmarks.landmark)
            mp_draw.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            

    # Display video feed
    cv2.imshow("Hand Tracking", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

# Release resources
pipeline.stop()
cv2.destroyAllWindows()


In [ ]:
num_coords = len(result.multi_hand_landmarks.landmark)
num_coords

In [ ]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [ ]:
connected_devices = [] # List of serial numbers for present cameras
for i in range(len(realsense_ctx.devices)):
        detected_camera =  realsense_ctx.devices[i].get_info(rs.camera_info.serial_number)
        print(f"{detected_camera}")
        connected_devices.append(detected_camera)
camera_input = input("Select camera: ")
if camera_input == '0':
        camera_id = connected_devices[0]
elif camera_input == '1':
        camera_id = connected_devices[1]
print(camera_input)

In [ ]:
file_name = f'coords_1_{camera_id}.csv'
file_path = os.path.join('/home/non/catkin_ws/src/robot_control/scripts/', file_name)
with open(file_path, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import os
import csv
from sklearn.cluster import KMeans
import pyrealsense2 as rs
import pandas as pd

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

font = cv2.FONT_HERSHEY_COMPLEX
org = (20, 100)
fontScale = 0.5
thickness = 1
color = (0, 150, 255)

# RealSense camera setup
realsense_ctx = rs.context()
connected_devices = [realsense_ctx.devices[i].get_info(rs.camera_info.serial_number) for i in range(len(realsense_ctx.devices))]

print("Connected Devices:", connected_devices)

camera_input = input("Select camera (0 or 1): ")
if camera_input.isdigit() and int(camera_input) < len(connected_devices):
    camera_id = connected_devices[int(camera_input)]
else:
    print("Invalid selection. Using default camera.")
    camera_id = connected_devices[0]

print("Using camera:", camera_id)

# Configure RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_device(camera_id)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
profile = pipeline.start(config)

align_to = rs.stream.color
align = rs.align(align_to)

depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
clipping_distance = 2 / depth_scale  # 2 meters

# Data Storage
collected_data = []
labels = []

def normalize_hand_keypoints(landmarks):
    """ Normalize hand keypoints based on the hand size and position """
    keypoints = np.array([[lm.x, lm.y, lm.z] for lm in landmarks.landmark])
    
    # Compute reference scale (wrist to index finger tip)
    wrist = keypoints[0]
    index_tip = keypoints[8]
    scale = np.linalg.norm(wrist - index_tip)  

    return ((keypoints - wrist) / scale).flatten()  # Flatten to 1D array

while True:
    frames = pipeline.wait_for_frames()
    aligned_frames = align.process(frames)
    depth_frame = aligned_frames.get_depth_frame()
    color_frame = aligned_frames.get_color_frame()

    if not depth_frame or not color_frame:
        continue

    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())
    color_image = cv2.flip(color_image, 1)

    image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)

    result = hands.process(image)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            keypoints = normalize_hand_keypoints(hand_landmarks)
            mp_draw.draw_landmarks(color_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            if cv2.waitKey(1) & 0xFF == ord('c'):
                class_name = input("Enter the class name: ")
                print(f"Captured data. Enter gesture label: {class_name}")
                collected_data.append(keypoints)
                labels.append(class_name)

    cv2.imshow("Hand Tracking", color_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Convert to NumPy array
collected_data = np.array(collected_data)
labels = np.array(labels)

if len(collected_data) > 0:
    kmeans = KMeans(n_clusters=min(len(collected_data), 100), random_state=42)
    cluster_labels = kmeans.fit_predict(collected_data)
    
    # Extract unique cluster centers
    filtered_data = kmeans.cluster_centers_
    filtered_labels = [labels[cluster_labels.tolist().index(i)] for i in range(len(kmeans.cluster_centers_))]

    file_name = f'coords_1_ds_{camera_id}.csv'
    file_path = os.path.join('/home/non/catkin_ws/src/robot_control/scripts/', file_name)

    with open(file_path, mode='a', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for i, row in enumerate(filtered_data):
            csv_writer.writerow(np.append(row, filtered_labels[i]))

    print(f"Data saved to {file_path}")
else:
    print("No data collected. Exiting.")

# Release resources
pipeline.stop()
cv2.destroyAllWindows()


Train LSTM

Train supervised

In [ ]:
df = pd.read_csv(f'coords_1_{camera_id}.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
np.shape(df)

In [ ]:
import pickle
import os
import logging
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Set up logging
logging.basicConfig(filename='trainingV3.log', level=logging.INFO)

In [ ]:
# Load dataset

X = df.drop('class', axis=1)  # Features
y = df['class']  # Target


In [ ]:
# Train-test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [ ]:
# Model Pipelines
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression(class_weight='balanced')),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
    'svm': make_pipeline(StandardScaler(), SVC(class_weight='balanced')),
    'knn': make_pipeline(StandardScaler(), KNeighborsClassifier())
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model
    
    # Save each model in .pkl and .pickle format
    with open(f'{algo}_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    with open(f'{algo}_model.pickle', 'wb') as f:
        pickle.dump(model, f)
    
    logging.info(f"Model {algo} trained and saved.")

/home/non/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Model evaluation using cross-validation
cv_results = {algo: cross_val_score(model, X_train, y_train, cv=5).mean() for algo, model in fit_models.items()}
logging.info(f"Cross-validation results: {cv_results}")

print(cv_results)  # Print cross-validation results

In [ ]:
# Save all models in a single file
with open('all_models.pkl', 'wb') as f:
    pickle.dump(fit_models, f)
with open('all_models.pickle', 'wb') as f:
    pickle.dump(fit_models, f)

Evaluate and  create confusion matrix

In [ ]:
# Generate and save confusion matrix for each model
for algo, model in fit_models.items():
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    
    # Save confusion matrix as an image
    cm_filename = f'{algo}_confusion_matrix.png'
    disp.figure_.savefig(cm_filename)
    logging.info(f"Confusion matrix for {algo} saved as {cm_filename}.")


Supervisied

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import pyrealsense2 as rs

# Load trained model # Change model before run
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# Initialize RealSense
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

# Function to normalize hand keypoints
def normalize_hand_keypoints(landmarks):
    keypoints = np.array([[lm.x, lm.y, lm.z] for lm in landmarks.landmark])

    wrist = keypoints[0]  # Wrist as reference point
    index_tip = keypoints[8]  # Index finger tip
    scale = np.linalg.norm(wrist - index_tip)  # Scale based on hand size

    keypoints = (keypoints - wrist) / scale  # Normalize
    return keypoints.flatten()  # Flatten to 1D array

# Start Real-Time Prediction
while True:
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    if not color_frame:
        continue

    color_image = np.asanyarray(color_frame.get_data())
    color_image = cv2.flip(color_image, 1)  # Flip for a mirror effect
    image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)

    # Process hand landmarks
    result = hands.process(image)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            keypoints = normalize_hand_keypoints(hand_landmarks)
            keypoints = keypoints.reshape(1, -1)  # Reshape for model input

            # Predict using trained model
            prediction_proba = model.predict_proba(keypoints)  # Get confidence scores
            predicted_class = model.predict(keypoints)[0]
            confidence = np.max(prediction_proba)  # Highest confidence score

            # Draw landmarks
            mp_draw.draw_landmarks(color_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Display prediction & confidence
            text = f'Prediction: {predicted_class} ({confidence:.2f})'
            text_color = (0, 255, 0) if confidence > 0.8 else (0, 255, 255)  # Green if confident, yellow if unsure

            cv2.putText(color_image, text, (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)

    else:
        # If no hand is detected
        cv2.putText(color_image, "No Hand Detected", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show video feed
    cv2.imshow("Hand Gesture Recognition", color_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
pipeline.stop()
cv2.destroyAllWindows()


NEW TRAININ METHOD, DO IT IF YOU WANT TO TRY BUT IT WILL TAKE SOME TIME

In [ ]:
import logging
import pickle
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
import skl2onnx
import onnxruntime as ort

# Set up logging
logging.basicConfig(filename='training_optimized.log', level=logging.INFO)

# Load dataset
df = pd.read_csv(f'coords_1_{camera_id}.csv') # Update with your actual dataset path



# Split dataset
X = df.drop('class', axis=1)
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# Handle class imbalance
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)

# Define models with hyperparameter tuning
param_grid = {
    'rf': {'n_estimators': [100, 200], 'max_depth': [10, None]},
    'svm': {'C': [1, 10], 'kernel': ['linear', 'rbf']},
    'knn': {'n_neighbors': [3, 5]}
}

pipelines = {
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'svm': make_pipeline(StandardScaler(), SVC(probability=True)),
    'knn': make_pipeline(StandardScaler(), KNeighborsClassifier())
}

best_models = {}
for algo, params in param_grid.items():
    grid_search = GridSearchCV(pipelines[algo], param_grid=params, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_models[algo] = grid_search.best_estimator_
    logging.info(f'Best {algo} model: {grid_search.best_params_}')

# Model stacking
stacking_model = StackingClassifier(
    estimators=[('rf', best_models['rf']), ('svm', best_models['svm']), ('knn', best_models['knn'])],
    final_estimator=LogisticRegression()
)
stacking_model.fit(X_train, y_train)

# Save models in .pkl and .pickle
joblib.dump(stacking_model, 'stacking_model.pkl')
with open('stacking_model.pickle', 'wb') as f:
    pickle.dump(stacking_model, f)

# Convert model to ONNX for faster inference
onnx_model = skl2onnx.convert_sklearn(stacking_model, initial_types=[('float_input', X_train[:1].astype(np.float32))])
with open('stacking_model.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())

# Model evaluation
y_pred = stacking_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=stacking_model.classes_, yticklabels=stacking_model.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()
